<a href="https://colab.research.google.com/github/AndresMontesDeOca/TimeSeries/blob/main/Unidad_2/AST_Clase_2_ejercicio_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block alert-info">
    <h1>Análisis de Series Temporales - 2023</h1>
    <h3>Clase 2</h3>
    <h5>Ejercicio 1</h5>
        <p>Docente: Rodrigo Del Rosso<p>
        <p>Asistentes: Sebastián Calcagno y Braian Drago <p>
</div>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.graphics.tsaplots as sgt

import warnings
warnings.filterwarnings('ignore')

Leemos los datos a trabajar


In [ ]:
url='https://raw.githubusercontent.com/sebcalcagno/AnalisisSeriesTemporales/main/series.csv'
data = pd.read_csv(url)

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.index = data['date']
del data['date']

In [ ]:
data=data.asfreq('d')
data.head()

In [ ]:
data.drop('x1', inplace=True, axis=1)
data.drop('x2', inplace=True, axis=1)
data.drop('x4', inplace=True, axis=1)
data.drop('x5', inplace=True, axis=1)
data.drop('x6', inplace=True, axis=1)

In [ ]:
data.rename({'x3': 'analisis'}, axis=1,inplace=True)

In [ ]:
result = type(data)
print(result)

In [ ]:
data.describe()

In [ ]:
sns.distplot(data.analisis,bins=20,color='#fcba03')
plt.title('Histograma de la Serie')

In [ ]:
import scipy.stats
import pylab

In [ ]:
scipy.stats.probplot(data.analisis, plot = pylab)
plt.title("QQ Plot", size = 24)
pylab.show()

In [ ]:
data.plot(figsize=(20,5), title = "Serie original")
plt.show()

In [ ]:
data_diff=data.copy()
data_diff=data_diff.diff(axis = 0, periods = 1)
data_diff.drop(['2021-11-28	'], inplace=True)

In [ ]:
data_diff.head()

In [ ]:
data_diff.analisis.plot(figsize=(20,5), title = "Serie diferenciada")
plt.show()

In [ ]:
plt.figure(figsize=(16, 8), dpi=110)

data['analisis'].plot(label='Serie original', color='orange')
data_diff['analisis'].plot(label='Serie diferenciada')

plt.title('Graficos de las series')

plt.xlabel('Tiempo')

plt.legend()

# Graficos serie original

In [ ]:
def autocovariance(series, lag):
    n = len(series)
    mean = np.mean(series)
    covar_sum = 0

    for t in range(lag, n):
        covar_sum += (series[t] - mean) * (series[t - lag] - mean)

    return covar_sum / (n - lag)

time_series = data.analisis

# Calcular autocovarianza para 20 retrasos (lags)
lags = range(20)

autocovars = [autocovariance(time_series, lag) for lag in lags]

# Graficar la función de autocovarianza
plt.bar(lags, autocovars)
plt.xlabel('Lags')
plt.ylabel('Autocovarianza')
plt.title('Funcion de Autocovarianza')
plt.show()

In [ ]:
sgt.plot_acf(data.analisis, zero = True, lags = 20)
plt.title("ACF para la serie original", size = 20)
plt.show()

*COMENTARIO NUEVO 1*

Una vez mas, vemos la grafica de la funcion de autocorrelacion que muestra cómo las observaciones en una serie de datos se relacionan entre sí a medida que se desplazan en el tiempo. Ayuda a identificar patrones de repetición en los datos.La relacion en cada momento es directa e indirecta con respecto al momento 0.
Con esta grafica, podemos llegar a tener una primera aproximacion de presencia o no de estacionariedad.

In [ ]:
sgt.plot_pacf(data.analisis, lags = 20, alpha = 0.05,  method = ('ols'))
plt.title("PACF para la serie original", size = 20)
plt.show()

*COMENTARIO NUEVO 2*

En este caso, la grafica es de una funcion de autocorrelacion parcial ( PACF = "Partial Autocorrelation Function ). La misma ayuda a identificar relaciones directas entre observaciones separadas en el tiempo, eliminando efectos indirectos. Cuando estamos trabajando con modelos autoregresivos, nos puede mostrar el orden del mismo.

# Graficos serie original diferenciada

In [ ]:
sgt.plot_acf(data_diff.analisis, zero = False, lags = 40)
plt.title("ACF para la diferencia de la serie original", size = 20)
plt.show()

In [ ]:
sgt.plot_pacf(data_diff.analisis, lags = 40, alpha = 0.05, zero = False)
plt.title("PACF de la diferencia de la serie original", size = 20)
plt.show()

# Modelo Autorregresivo de orden 1

In [ ]:
from statsmodels.tsa.arima_model import ARMA
import statsmodels.api as sm

In [ ]:
model = sm.tsa.arima.ARIMA(data.analisis, order=((1),0,0))
result = model.fit()

In [ ]:
result.summary()

In [ ]:
result.arroots

In [ ]:
print(f"La raiz REAL del modelo condicionado a un AR(1)  es= {result.arroots} \n vemos que es en modulo mayor a 1. Es decir, es estacionaria ")

In [ ]:
print(f"Los parametros que arroja el modelo son= \n {result.params} ")

In [ ]:
print(f"Los criterios de informacion que arroja el modelo son \n -Akaike= {result.aic} \n -Bayesiano= {result.bic} \n -Hannan-Quinn {result.hqic}")

Presentacion de los residuos del modelo

In [ ]:
print(f" La media de los residuos del modelo es=  {result.resid.mean()}")

In [ ]:
print(f" La varianza de los residuos del modelo es=  {result.resid.var()}")

In [ ]:
result.resid.plot(figsize=(20,5))
plt.title("Residuos del modelo",size=24)
plt.show()

*COMENTARIO NUEVO 3*

En el gráfico, se presentan los residuos del modelo. Para validar la elección adecuada del modelo, es fundamental que los resultados de estos residuos cumplan con la propiedad de "ruido blanco". Es recomendable enfatizar este concepto y aclarar cualquier pregunta o inquietud que puedan surgir al respecto. Estoy a disposición para responder cualquier consulta sin ningún problema.

In [ ]:
sns.distplot(result.resid,bins=12,color='#fcba03')
plt.title('Histograma de los residuos del modelo')

In [ ]:
import scipy.stats
import pylab

In [ ]:
scipy.stats.probplot(result.resid, plot = pylab)
plt.title("QQ Plot residuos del modelo", size = 24)
pylab.show()

*Prueba* Ljung-Box

In [ ]:
import statsmodels.api as sm

In [ ]:
Incorr_residuos_modelo=sm.stats.acorr_ljungbox(result.resid, lags=30)

In [ ]:
Incorr_residuos_modelo

In [ ]:
plt.plot(Incorr_residuos_modelo.lb_pvalue, marker='o', markersize=10, linestyle='--', linewidth=1)
plt.title("Grafico de incorrelacion de los residuos del modelo")
plt.ylabel("P-valores ")

*COMENTARIO NUEVO 4*

Aqui vemos la grafica de incorrelacion de los residuos del modelo, los residuos tienen q estar incorrelacionados. Tienen q ser proxi al ruido blanco. Una forma de analisis es esta grafica para ver esto. En este caso, los residuos no presentan correlacion.